In [ ]:
CR_count<-function(ID, fastq_path, organism, localcores_= 5, mem_percent = 0.05, kwargs=''){
    ### R wrapper for cell ranger count function
    ### simple gene expression no feature barcodes
    ### kwargs generated using hash, all key:value pairs must be strings
    ### e.g. kwargs = hash(keys = list('expect-cells', 'force-cells'), values = list('1000', '800'))
    ###
    ### ID is name of directory to be saved in 
    ### fastq_path is path/to/fastq_directory
    ### functionality right now is that output is in directory that script is run in, 
    ### unsure how to change this as ID does not take a path/to/id_name, just an id_name
    
    # get organisms reference transcriptome
    if(organism == 'human'){transcriptome_ = '/data/hratch/Software/refdata-cellranger-GRCh38-3.1.0/GRCh38'}
    else if (organism == 'mouse'){transcriptome_ = '/data/hratch/Software/refdata-cellranger-mm10-3.1.0/mm10'}
    else{
        msg<-'Download appropriate reference transcriptome in /data/hratch/Software/ and update'
        msg<-paste0(msg, ' CellRanger_count_wrapper.r script in general_analyses')
        stop(msg)}
    
    # get total available memory in GB
    mem = strsplit(system('free -h', intern=TRUE), ' ')[[2]]
    mem = mem[length(mem)]
    mem = strsplit(mem, 'G')[[1]]
    # set mem to mem_percent of total available memory
    mem = (as.integer(as.double(mem)*mem_percent))
    if (mem > 128){mem = 128}
    mem = as.character(mem)
    
    str = paste0('cellranger count --id=', ID, ' --fastqs=', fastq_path, ' --transcriptome=', transcriptome_,
                ' --localcores=', localcores_, ' --localmem=', mem)
        
    # add kwargs
    if(is.hash(kwargs)){for(key in keys(kwargs)){str=paste0(str, ' --', key, '=', kwargs[[key]])}}
#     return(str)
    system(str, intern = TRUE)
    
}

In [ ]:
local_data_path = '/data2/hratch/STAT4/'

In [ ]:
print('Run cell ranger count on patient/control fastqs')
# run in /data2/hratch/STAT4/interim/
samples = list('5k_pbmc_v3_fastqs', 'LB441_fastqs', 'LB443_fastqs')
index = 1
for(sample in samples){
    print(sample)
    fastq_path = paste0(local_data_path, 'raw/', sample)
    ID = paste0(strsplit(sample, 'fastqs'), 'count')

    # run CellRanger count function
    if(sample == '5k_pbmc_v3_fastqs'){ # for control, expected cells is 5k according to title
        CR_count(ID, fastq_path, organism = 'human', localcores_=8, mem_percent = 0.1,
                 kwargs = hash(keys = list('expect-cells'), values = list('5000')))
    }
    else{CR_count(ID, fastq_path, organism = 'human', localcores_=8, mem_percent = 0.1)}


    index<-index + 1
}